In [1]:
# upload dataset
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-12-11 15:11:05--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-12-11 15:11:06--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb1bce0516f3c3d5ba56f636ecd.dl.dropboxusercontent.com/cd/0/inline/Bybd0SZs0R92qPBjO-0H6Hhtwppns5iFGgqEZd9D-KeNaZmtIYaTnSIOxEaiE5V4TEqjhYemcqp5DLvbgluuzeFPAbYlJbMdVJHGOVAT0qqp4dbhs7mvz7nwRq3Al9m7jZAaY9hw3GSn16AmpzYf_RDLMj8YnvMpAVRQy0PuRik9vA/file# [following]
--2022-12-11 15:11:06--  https://ucb1bce0516f3c3d5ba56f636ecd.dl.dropboxusercontent.com/cd/0/inline/Bybd0SZs0R92qPBjO-0H6Hhtwppns5iFGgqEZd

In [3]:
# unzip file 
! unzip -q "/content/movie-reviews-dataset.zip"

In [4]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout, Bidirectional

In [9]:
# prepar Data
def prepareData(dir):
    data = text_dataset_from_directory(dir)
    return data.map(
        lambda text, label: (regex_replace(text, 'br />', ''), label),
    )

In [10]:
# split data
train_data = prepareData("movie-reviews-dataset/train")
test_data = prepareData("movie-reviews-dataset/test")

for text_batch, label_batch in train_data.take(1):
    print(text_batch.numpy()[0])
    print(label_batch.numpy()[0])


Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"First of all.....<<What the hell? Why in the world are they trying to sell a low budget piece of crap on late night TV with the promise of disturbing, offensive sick garbage that any normal real human being with a soul couldn't watch. <<What the crap is funny about a dog being injured, a grandma getting her head knocked off...a guy getting his hand blown to pieces and two girls going to the bathroom? what in the hell has this world come to that people find comedy in some thing so completely sick. Anyone who thinks this kind of material is funny, should not even be allowed to walk the earth. <<But from what I hear its not even offensive...so...they promise comedy through demented piles of sick garbage...and they cant even pull that off."
0


In [11]:
# model 
model = Sequential()
model.add(Input(shape=(1,), dtype="string"))

In [16]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

In [17]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [18]:
model.add(Embedding(max_tokens + 1, 128))

rnn = RNN(SimpleRNNCell(64), return_sequences=False, return_state=False)
model.add(Bidirectional(GRU(64)))
model.add(rnn)
model.add(Dense(1, activation="sigmoid"))

In [19]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 105s 128ms/step - loss: 0.5462 - accuracy: 0.7100
Epoch 2/10
782/782 [==============================] - 101s 129ms/step - loss: 0.4335 - accuracy: 0.7994
Epoch 3/10
782/782 [==============================] - 101s 130ms/step - loss: 0.3937 - accuracy: 0.8208
Epoch 4/10
782/782 [==============================] - 101s 130ms/step - loss: 0.3628 - accuracy: 0.8364
Epoch 5/10
782/782 [==============================] - 102s 130ms/step - loss: 0.3341 - accuracy: 0.8523
Epoch 6/10
782/782 [==============================] - 103s 132ms/step - loss: 0.3040 - accuracy: 0.8658
Epoch 7/10
782/782 [==============================] - 101s 129ms/step - loss: 0.2751 - accuracy: 0.8804
Epoch 8/10
782/782 [==============================] - 103s 132ms/step - loss: 0.2431 - accuracy: 0.8972
Epoch 9/10
782/782 [==============================] - 101s 128ms/step - loss: 0.2149 - accuracy: 0.9082
Epoch 10/10
782/782 [==============================] - 102s 130m

In [22]:
model.evaluate(test_data)

782/782 [==============================] - 31s 39ms/step - loss: 0.7387 - accuracy: 0.7666


[0.7386775612831116, 0.7666000127792358]

In [23]:
text = "I loved the movie !"

In [24]:
model.predict([text])

1/1 [==============================] - 1s 829ms/step


array([[0.9984604]], dtype=float32)